<a href="https://colab.research.google.com/github/farhadkamangar/CSE5368/blob/master/tensorflow_saving_model_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
from datetime import datetime

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
class FlexibleDenseLayer(tf.Module):
  def __init__(self, number_of_nodes, name=None):
    super().__init__(name=name)
    self.first_time=True
    self.number_of_nodes=number_of_nodes
  @tf.function  
  def calculate_output(self, x):
    if self.first_time:
      self.w = tf.Variable(tf.random.normal([x.shape[-1], self.number_of_nodes]), name='weights')
      self.b = tf.Variable(tf.zeros([self.number_of_nodes]), name='biases')
      self.first_time=False
    y = tf.matmul(x, self.w) + self.b
    # return tf.nn.hard_tanh(y)
    # return tf.nn.relu(y)
    return y

In [22]:
class MultiLayer_NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.layer_1 = FlexibleDenseLayer(number_of_nodes=3)
    self.layer_2 = FlexibleDenseLayer(number_of_nodes=5)
  @tf.function
  def predict(self, x):
    x = self.layer_1.calculate_output(x)
    return self.layer_2.calculate_output(x)

my_model = MultiLayer_NN(name="the_model")

output_of_model=my_model.predict(tf.constant([[1.0, 2.0, 3.0]]))
print("Model results:", output_of_model)
print("Weights ", my_model.layer_2.w)
print("Submodules:", my_model.submodules)
for var in my_model.variables:
  print(var, "\n")

Model results: tf.Tensor([[-0.07391735  1.553375    0.93301284  1.0583192  -2.8184297 ]], shape=(1, 5), dtype=float32)
Weights  <tf.Variable 'weights:0' shape=(3, 5) dtype=float32, numpy=
array([[-1.2272114 , -1.2106657 ,  0.5175059 ,  0.48413256, -1.2170948 ],
       [ 0.31798697,  0.95185465,  0.16022263,  1.6761829 , -2.250372  ],
       [ 0.05292824,  1.7372127 ,  0.9420731 ,  0.34149462, -2.0420725 ]],
      dtype=float32)>
Submodules: (<__main__.FlexibleDenseLayer object at 0x7ffb3722a0d0>, <__main__.FlexibleDenseLayer object at 0x7ffb3722a450>)
<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.0808226 ,  0.66871613,  3.188913  ],
       [ 0.1440099 ,  0.7798974 , -0.7055703 ],
       [-0.38929355, -0.6067691 , -0.32240286]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0

In [23]:
tf.saved_model.save(my_model, "saved_model_02")

In [24]:
ls -l saved_model_02

total 28
drwxr-xr-x 2 root root  4096 Oct 18 00:35 assets/
-rw-r--r-- 1 root root 16959 Oct 18 00:38 saved_model.pb
drwxr-xr-x 2 root root  4096 Oct 18 00:38 variables/


In [25]:
ls -l saved_model_02/variables

total 8
-rw-r--r-- 1 root root 628 Oct 18 00:38 variables.data-00000-of-00001
-rw-r--r-- 1 root root 357 Oct 18 00:38 variables.index


In [26]:
my_model_02 = tf.saved_model.load("saved_model_02")
output_of_model=my_model_02.predict(tf.constant([[1.0, 2.0, 3.0]]))
print("Model results:", output_of_model)

Model results: tf.Tensor([[-0.07391735  1.553375    0.93301284  1.0583192  -2.8184297 ]], shape=(1, 5), dtype=float32)
